In [1]:
import mimicgen
import argparse
import datetime
import json
import os
import shutil
import time
from glob import glob

import h5py
import numpy as np

import robosuite as suite
import robosuite.macros as macros
from robosuite import load_controller_config
# from robosuite.utils.input_utils import input2action
from robosuite.wrappers import DataCollectionWrapper, VisualizationWrapper

[robosuite WARNING] No private macro file found! (macros.py:56)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:57)
[robosuite WARNING] To setup, run: python /home/ns/mimicgen/mimicgen_envs/robosuite/robosuite/scripts/setup_macros.py (macros.py:58)


In [2]:
dir = "/tmp/1735324586_3655322/" 

In [3]:
dirs=os.listdir(dir)
len(dirs)

100

In [4]:
for d in dirs:
    fp=dir+d 
    files=os.listdir(fp)
    print(fp, len(files))

/tmp/1735324586_3655322/ep_1735325479_7619808 6
/tmp/1735324586_3655322/ep_1735325597_3093696 6
/tmp/1735324586_3655322/ep_1735325625_8191903 7
/tmp/1735324586_3655322/ep_1735324787_7405546 6
/tmp/1735324586_3655322/ep_1735324853_1674597 7
/tmp/1735324586_3655322/ep_1735325463_502039 7
/tmp/1735324586_3655322/ep_1735325610_2238033 7
/tmp/1735324586_3655322/ep_1735325558_0972912 6
/tmp/1735324586_3655322/ep_1735325666_8580472 6
/tmp/1735324586_3655322/ep_1735325722_2117093 7
/tmp/1735324586_3655322/ep_1735324808_9896336 6
/tmp/1735324586_3655322/ep_1735325818_8975248 5
/tmp/1735324586_3655322/ep_1735324664_7712777 6
/tmp/1735324586_3655322/ep_1735324639_06119 6
/tmp/1735324586_3655322/ep_1735324756_9741948 5
/tmp/1735324586_3655322/ep_1735324999_9656456 6
/tmp/1735324586_3655322/ep_1735324627_6443233 6
/tmp/1735324586_3655322/ep_1735325693_8449147 6
/tmp/1735324586_3655322/ep_1735325316_1697772 6
/tmp/1735324586_3655322/ep_1735325037_2013516 6
/tmp/1735324586_3655322/ep_1735325892_32307

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--directory",
    type=str,
    default=os.path.join(suite.models.assets_root, "demonstrations"),
)
parser.add_argument("--environment", type=str, default="Lift")
parser.add_argument("--robots", nargs="+", type=str, default="Panda", help="Which robot(s) to use in the env")
parser.add_argument(
    "--config", type=str, default="single-arm-opposed", help="Specified environment configuration if necessary"
)
parser.add_argument("--arm", type=str, default="right", help="Which arm to control (eg bimanual) 'right' or 'left'")
parser.add_argument("--camera", type=str, default="agentview", help="Which camera to use for collecting demos")
parser.add_argument(
    "--controller", type=str, default="OSC_POSE", help="Choice of controller. Can be 'IK_POSE' or 'OSC_POSE'"
)
parser.add_argument("--device", type=str, default="keyboard")
parser.add_argument("--pos-sensitivity", type=float, default=1.0, help="How much to scale position user inputs")
parser.add_argument("--rot-sensitivity", type=float, default=1.0, help="How much to scale rotation user inputs")
parser.add_argument("--success-only", action="store_true", help="Only save successful demos", default=False)
args = parser.parse_args(["--device", "spacemouse", "--environment", "Coffee", "--rot-sensitivity","0.8"])

In [6]:
# Get controller config
controller_config = load_controller_config(default_controller=args.controller)

# Create argument configuration
config = {
    "env_name": args.environment,
    "robots": args.robots,
    "controller_configs": controller_config,
}

# Check if we're using a multi-armed environment and use env_configuration argument if so
if "TwoArm" in args.environment:
    config["env_configuration"] = args.config

# Create environment
env = suite.make(
    **config,
    has_renderer=True,
    has_offscreen_renderer=False,
    render_camera=args.camera,
    ignore_done=True,
    use_camera_obs=False,
    reward_shaping=True,
    control_freq=20,
)

# Wrap this with visualization wrapper
env = VisualizationWrapper(env)

# Grab reference to controller config and convert it to json-encoded string
env_info = json.dumps(config)

In [7]:
def gather_demonstrations_as_hdf5(save_count, directory, out_dir, env_info, stopped=False, successonly=False):
    """ 
    Args:
        directory (str): Path to the directory containing raw demonstrations.
        out_dir (str): Path to where to store the hdf5 file.
        env_info (str): JSON-encoded string containing environment information,
            including controller and robot info
    """

    hdf5_path = os.path.join(out_dir, "demo.hdf5")
    f = h5py.File(hdf5_path, "w")

    # store some metadata in the attributes of one group
    grp = f.create_group("data")

    num_eps = 0
    env_name = None  # will get populated at some point

    for ep_directory in os.listdir(directory):

        state_paths = os.path.join(directory, ep_directory, "state_*.npz")
        states = []
        actions = []
        success = False

        for state_file in sorted(glob(state_paths)):
            dic = np.load(state_file, allow_pickle=True)
            env_name = str(dic["env"])

            states.extend(dic["states"])
            for ai in dic["action_infos"]:
                actions.append(ai["actions"])
            success = success or dic["successful"]

        if len(states) == 0:
            continue

        if successonly and not success:
            continue
 
        print(f"Demonstration is successful? {success} and has been saved")
        # Delete the last state. This is because when the DataCollector wrapper
        # recorded the states and actions, the states were recorded AFTER playing that action,
        # so we end up with an extra state at the end.
        del states[-1]
        assert len(states) == len(actions)

        
        ep_data_grp = grp.create_group("demo_{}".format(num_eps))
        num_eps += 1
        

        # store model xml as an attribute
        xml_path = os.path.join(directory, ep_directory, "model.xml")
        with open(xml_path, "r") as f:
            xml_str = f.read()
        ep_data_grp.attrs["model_file"] = xml_str

        # write datasets for states and actions
        ep_data_grp.create_dataset("states", data=np.array(states))
        ep_data_grp.create_dataset("actions", data=np.array(actions))
 

    # write dataset attributes (metadata)
    now = datetime.datetime.now()
    grp.attrs["date"] = "{}-{}-{}".format(now.month, now.day, now.year)
    grp.attrs["time"] = "{}:{}:{}".format(now.hour, now.minute, now.second)
    grp.attrs["repository_version"] = suite.__version__
    grp.attrs["env"] = env_name
    grp.attrs["env_info"] = env_info

    f.close()
    return save_count+1

In [8]:
save_count=gather_demonstrations_as_hdf5(0, dir, dir, env_info, successonly=args.success_only)

Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been saved
Demonstration is successful? True and has been